# _LangChain_ Ensemble Retriever

The _LangChain_ EnsembleRetriever takes a list of retrievers as input and ensemble the results of their get_relevant_documents() methods and rerank the results based on the Reciprocal Rank Fusion algorithm. With TruLens, we have the ability to evaluate the context of each component retriever along with the ensemble retriever. This example walks through that process.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/langchain/langchain_ensemble_retriever.ipynb)

## Setup

In [ ]:
# ! pip install trulens_eval openai langchain langchain_community langchain_openai rank_bm25 faiss_cpu

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [2]:
# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
from trulens_eval.schema.feedback import FeedbackResult
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

/opt/anaconda3/envs/snowday/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

## Initialize Context Relevance checks for each component retriever + ensemble

This requires knowing the feedback selector for each. You can find this path by logging a run of your application and examining the application traces on the Evaluations page.

Read more in our docs: https://www.trulens.org/trulens_eval/selecting_components/

In [4]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.schema.feedback import Select
import numpy as np

# Initialize provider class
openai = OpenAI()

bm25_context = Select.RecordCalls.retrievers[0]._get_relevant_documents.rets[:].page_content
faiss_context = Select.RecordCalls.retrievers[1]._get_relevant_documents.rets[:].page_content
ensemble_context = Select.RecordCalls.invoke.rets[:].page_content

# Question/statement relevance between question and each context chunk.
f_context_relevance_bm25 = (
    Feedback(openai.context_relevance, name = "BM25")
    .on_input()
    .on(bm25_context)
    .aggregate(np.mean)
    )

f_context_relevance_faiss = (
    Feedback(openai.context_relevance, name = "FAISS")
    .on_input()
    .on(faiss_context)
    .aggregate(np.mean)
    )

f_context_relevance_ensemble = (
    Feedback(openai.context_relevance, name = "Ensemble")
    .on_input()
    .on(ensemble_context)
    .aggregate(np.mean)
    )

✅ In BM25, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In BM25, input context will be set to __record__.app.retrievers[0]._get_relevant_documents.rets[:].page_content .
✅ In FAISS, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In FAISS, input context will be set to __record__.app.retrievers[1]._get_relevant_documents.rets[:].page_content .
✅ In Ensemble, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ensemble, input context will be set to __record__.app.invoke.rets[:].page_content .


## Add feedbacks

In [5]:
tru_recorder = TruChain(ensemble_retriever,
    app_id='Ensemble Retriever',
    feedbacks=[f_context_relevance_bm25, f_context_relevance_faiss, f_context_relevance_ensemble])

In [6]:
with tru_recorder as recording:
    ensemble_retriever.invoke("apples")

## See and compare results from each retriever

In [7]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, 'Ensemble')

,question,context,ret
0,apples,I like apples,1.0
1,apples,You like apples,1.0
2,apples,Apples and oranges are fruits,0.4
3,apples,You like oranges,0.0


In [8]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, 'BM25')

,question,context,ret
0,apples,I like apples,1.0
1,apples,Apples and oranges are fruits,1.0


In [9]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, 'FAISS')

,question,context,ret
0,apples,You like apples,1.0
1,apples,You like oranges,0.0


## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.